In [1]:
import itertools
import pandas as pd
from scipy.stats import norm

# some data im trying to fit with a gaussian distribution
data = [0.3, 0.1, 0.5] 

# the potential means, in an actual application there would be more intervals in the discretization
mu = [-1, 0, 1]
# the potential standard deviations.
sigma = [1, 2, 3]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# lets make a table of all the hypothesis and their prior distributions to keep track of what is happening

combinations = list(itertools.product(mu, sigma)) # total of 9 hypothesis
priors = [1/9] * 9
df = pd.DataFrame()
df['hypothesis (mean, std)'] = combinations
df['prior'] = priors

df

,"hypothesis (mean, std)",prior
0,"(-1, 1)",0.111111
1,"(-1, 2)",0.111111
2,"(-1, 3)",0.111111
3,"(0, 1)",0.111111
4,"(0, 2)",0.111111
5,"(0, 3)",0.111111
6,"(1, 1)",0.111111
7,"(1, 2)",0.111111
8,"(1, 3)",0.111111


In [3]:
def compute_likelihood(hyps, d):
    likelihoods = []
    for (mu, sigma) in hyps:
        likelihood = norm.pdf(d, mu, sigma)
        likelihoods.append(likelihood)
    return likelihoods

In [4]:
# compute the likelihood for first data point
df['likelihoods_1'] = compute_likelihood(df['hypothesis (mean, std)'].tolist(), data[0])
df['unnormalized_posterior_1'] = df['prior'] * df['likelihoods_1']
df

,"hypothesis (mean, std)",prior,likelihoods_1,unnormalized_posterior_1
0,"(-1, 1)",0.111111,0.171369,0.019041
1,"(-1, 2)",0.111111,0.161486,0.017943
2,"(-1, 3)",0.111111,0.121064,0.013452
3,"(0, 1)",0.111111,0.381388,0.042376
4,"(0, 2)",0.111111,0.197240,0.021916
5,"(0, 3)",0.111111,0.132318,0.014702
6,"(1, 1)",0.111111,0.312254,0.034695
7,"(1, 2)",0.111111,0.187620,0.020847
8,"(1, 3)",0.111111,0.129410,0.014379


In [5]:
# compute the total probability of the data, p(D), which is the sum of the unnormalized posterior
total_prob = df['unnormalized_posterior_1'].sum()
total_prob

0.19934966408579136

In [6]:
# normalize the posterior
df['posterior_1'] = df['unnormalized_posterior_1'] / total_prob
df

,"hypothesis (mean, std)",prior,likelihoods_1,unnormalized_posterior_1,posterior_1
0,"(-1, 1)",0.111111,0.171369,0.019041,0.095515
1,"(-1, 2)",0.111111,0.161486,0.017943,0.090007
2,"(-1, 3)",0.111111,0.121064,0.013452,0.067477
3,"(0, 1)",0.111111,0.381388,0.042376,0.212573
4,"(0, 2)",0.111111,0.197240,0.021916,0.109935
5,"(0, 3)",0.111111,0.132318,0.014702,0.073750
6,"(1, 1)",0.111111,0.312254,0.034695,0.174040
7,"(1, 2)",0.111111,0.187620,0.020847,0.104573
8,"(1, 3)",0.111111,0.129410,0.014379,0.072129


In [7]:
# do that again a couple times with the rest of the data, taking the previous posterior as the new prior
df['likelihoods_2'] = compute_likelihood(df['hypothesis (mean, std)'].tolist(), data[1])
df['unnormalized_posterior_2'] = df['posterior_1'] * df['likelihoods_2']
total_prob = df['unnormalized_posterior_2'].sum()
df['posterior_2'] = df['unnormalized_posterior_2'] / total_prob
df

,"hypothesis (mean, std)",prior,likelihoods_1,unnormalized_posterior_1,posterior_1,likelihoods_2,unnormalized_posterior_2,posterior_2
0,"(-1, 1)",0.111111,0.171369,0.019041,0.095515,0.217852,0.020808,0.088528
1,"(-1, 2)",0.111111,0.161486,0.017943,0.090007,0.171472,0.015434,0.065662
2,"(-1, 3)",0.111111,0.121064,0.013452,0.067477,0.124335,0.008390,0.035694
3,"(0, 1)",0.111111,0.381388,0.042376,0.212573,0.396953,0.084382,0.358999
4,"(0, 2)",0.111111,0.197240,0.021916,0.109935,0.199222,0.021901,0.093179
5,"(0, 3)",0.111111,0.132318,0.014702,0.073750,0.132907,0.009802,0.041702
6,"(1, 1)",0.111111,0.312254,0.034695,0.174040,0.266085,0.046310,0.197023
7,"(1, 2)",0.111111,0.187620,0.020847,0.104573,0.180263,0.018851,0.080200
8,"(1, 3)",0.111111,0.129410,0.014379,0.072129,0.127129,0.009170,0.039012


In [70]:
df['likelihoods_3'] = compute_likelihood(df['hypothesis (mean, std)'].tolist(), data[2])
df['unnormalized_posterior_3'] = df['posterior_2'] * df['likelihoods_3']
total_prob = df['unnormalized_posterior_3'].sum()
df['posterior_3'] = df['unnormalized_posterior_3'] / total_prob
df

,"hypothesis (mean, std)",prior,likelihoods_1,unnormalized_posterior_1,posterior_1,likelihoods_2,unnormalized_posterior_2,posterior_2,likelihoods_3,unnormalized_posterior_3,posterior_3
0,"(-1, 1)",0.111111,0.217852,0.024206,0.119948,0.171369,0.020555,0.088528,0.129518,0.011466,0.043202
1,"(-1, 2)",0.111111,0.171472,0.019052,0.094411,0.161486,0.015246,0.065662,0.150569,0.009887,0.037252
2,"(-1, 3)",0.111111,0.124335,0.013815,0.068458,0.121064,0.008288,0.035694,0.117355,0.004189,0.015783
3,"(0, 1)",0.111111,0.396953,0.044106,0.218560,0.381388,0.083356,0.358999,0.352065,0.126391,0.476223
4,"(0, 2)",0.111111,0.199222,0.022136,0.109691,0.197240,0.021635,0.093179,0.193334,0.018015,0.067877
5,"(0, 3)",0.111111,0.132907,0.014767,0.073178,0.132318,0.009683,0.041702,0.131147,0.005469,0.020606
6,"(1, 1)",0.111111,0.266085,0.029565,0.146505,0.312254,0.045747,0.197023,0.352065,0.069365,0.261357
7,"(1, 2)",0.111111,0.180263,0.020029,0.099252,0.187620,0.018622,0.080200,0.193334,0.015505,0.058422
8,"(1, 3)",0.111111,0.127129,0.014125,0.069997,0.129410,0.009058,0.039012,0.131147,0.005116,0.019278


In [71]:
# the final posterior looks like this
df[['hypothesis (mean, std)', 'posterior_3']]

,"hypothesis (mean, std)",posterior_3
0,"(-1, 1)",0.043202
1,"(-1, 2)",0.037252
2,"(-1, 3)",0.015783
3,"(0, 1)",0.476223
4,"(0, 2)",0.067877
5,"(0, 3)",0.020606
6,"(1, 1)",0.261357
7,"(1, 2)",0.058422
8,"(1, 3)",0.019278


In [ ]:
# so mu = 0, std = 1 were the most likely parameters.